<a href="https://colab.research.google.com/github/Rajesh1215/Danger_text_classification_Bert_model/blob/main/Tweet_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle competitions download -c nlp-getting-started

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/nlp-getting-started.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Flatten,Dropout,Input
import re

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
ss = pd.read_csv("sample_submission.csv")

In [ ]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [ ]:
train["keyword"].value_counts()

fatalities               45
deluge                   42
armageddon               42
sinking                  41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [ ]:
#Define a dictionary of common contractions and their expanded forms
contraction_dict = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "could've": "could have",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "I'd": "I would",
    "I'll": "I will",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "might've": "might have",
    "must've": "must have",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'll": "we will",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what's": "what is",
    "won't": "will not",
    "would've": "would have",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have",
    "I'd've": "I would have",
    "she'd've": "she would have",
    "should've": "should have",
    "could've": "could have",
    "might've": "might have",
    "would've": "would have"
    # Add more contractions and their expansions as needed
}

# Function to expand contractions using the dictionary
def expand_contractions(text):
    for contraction, expansion in contraction_dict.items():
        text = text.replace(contraction, expansion)
    return text


In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained(model_name)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
def cleaner(text):
  text=text["text"]
  emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F700-\U0001F77F"  # alchemical symbols
                           u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                           u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                           u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                           u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                           u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                           u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing
                           u"\U0001F004"             # Mahjong Tile Red Dragon
                           u"\U0001F0CF"             # Playing Cards Joker
                           u"\U0001F170-\U0001F251"  # Enclosed Ideographic Supplement
                           u"\U0001F300-\U0001F320"  # Miscellaneous Symbols and Pictographs
                           u"\U0001F321-\U0001F32C"  # Enclosed CJK Letters and Months
                           "]+", flags=re.UNICODE)
  text = re.sub(emoji_pattern, '', text)
  text = text.lower()
  text = re.sub(r"http\S+", "webadress", text)
  text = re.sub(r"<.*?>", " ", text)
  text = re.sub(r"#", "", text)
  text = re.sub(r'\w+@\S+', "emailadress", text)
  text = re.sub(r'[^a-zA-Z0-9]', " ", text)
  text=expand_contractions(text)
  return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

In [ ]:

X_train = train.apply(cleaner, axis=1)
X_text = test.apply(cleaner,axis=1)

In [ ]:
Y_train =  train.loc[:,"target"]

In [ ]:
X_train=np.array(X_train)
Y_train=np.array(Y_train)

In [ ]:

# Load the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load the DistilBERT model for sequence classification
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Input layers
input_word_ids = Input(shape=(512,), dtype=tf.int32, name="input_word_ids")
input_mask = Input(shape=(512,), dtype=tf.int32, name="input_mask")

# Get the logits from the DistilBERT model
logits = distilbert_model([input_word_ids, input_mask])[0]

# Add a classification layer
clf_output = Dense(1, activation='sigmoid')(logits)

# Create and compile the model
model = Model(inputs=[input_word_ids, input_mask], outputs=clf_output)
model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'], run_eagerly=True)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# Assuming X_train_tokens is a list of BatchEncoding objects
# Convert to tensors
X_train_ids = tf.squeeze(tf.convert_to_tensor([x["input_ids"] for x in X_train]), axis=[1])
X_train_mask = tf.squeeze(tf.convert_to_tensor([x["attention_mask"] for x in X_train]), axis=[1])

# Assuming Y_train is a NumPy array or list
Y_train = tf.convert_to_tensor(Y_train, dtype=tf.float32)

# Define a smaller batch size (e.g., 32)
small_batch_size = 32

# Now you can use X_train_ids, X_train_mask, and Y_train to fit your model with the reduced batch size
model.fit([X_train_ids, X_train_mask], Y_train, epochs=5, validation_split=0.1, batch_size=small_batch_size)


Epoch 1/5


ResourceExhaustedError: ignored

In [ ]:
!pip install tensorflow_text

In [ ]:
!pip install tensorflow_hub

In [ ]:
!pip install tensorflow_addons

In [ ]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow_addons as tfa

# Sample text data and labels (replace with your own data)
text_data = train["text"]
labels = train["target"]

# Load the BERT model from TensorFlow Hub
bert_model_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
bert_layer = hub.KerasLayer(bert_model_url, trainable=True)

# Tokenize the text data
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
outputs = bert_layer(encoder_inputs)

# Add a classification head
pooled_output = outputs["pooled_output"]
output = tf.keras.layers.Dropout(0.1)(pooled_output)
output = tf.keras.layers.Dense(1, activation="sigmoid")(output)

model = tf.keras.Model(text_input, output)

# Compile the model
optimizer = tfa.optimizers.RectifiedAdam(learning_rate=1e-5)
loss = tf.keras.losses.BinaryCrossentropy()
metric = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Tokenize and preprocess the text data
X_train = text_data
y_train = labels

# Fit the model
model.fit(X_train, y_train, epochs=5, batch_size=32,validation_split=0.1)


Epoch 1/5
215/215 [==============================] - 223s 931ms/step - loss: 0.5777 - binary_accuracy: 0.6936 - val_loss: 0.4138 - val_binary_accuracy: 0.8241
Epoch 2/5
215/215 [==============================] - 199s 923ms/step - loss: 0.4077 - binary_accuracy: 0.8259 - val_loss: 0.4180 - val_binary_accuracy: 0.8307
Epoch 3/5
215/215 [==============================] - 199s 925ms/step - loss: 0.3614 - binary_accuracy: 0.8529 - val_loss: 0.4348 - val_binary_accuracy: 0.8281
Epoch 4/5
215/215 [==============================] - 195s 909ms/step - loss: 0.3308 - binary_accuracy: 0.8705 - val_loss: 0.3783 - val_binary_accuracy: 0.8346
Epoch 5/5
215/215 [==============================] - 198s 922ms/step - loss: 0.2962 - binary_accuracy: 0.8869 - val_loss: 0.4162 - val_binary_accuracy: 0.8386


In [ ]:
test_tar=model.predict(test["text"])

102/102 [==============================] - 33s 321ms/step


In [ ]:
model.save("Text_classification_danger_or_not.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
res=model.predict(["help me.i am stuck in a lions cave its eating me i am dying"])[0][0]
if (res>0.5):
  print('Danger')
else:
  print('No Danger')

1/1 [==============================] - 0s 60ms/step
Danger


In [ ]:
import pandas as pd

# Create a sample DataFrame
#test_tar = {'Value': [0.3, 0.7, 0.8, 0.2]}

df = ss.copy()
test_tar=pd.DataFrame(test_tar)
# Apply a lambda function to convert values to 1 if greater than 0.5, else 0, row-wise (axis=1)

test_tar

,0
0,0.966388
1,0.982420
2,0.990326
3,0.993273
4,0.982190
...,...
3258,0.528103
3259,0.994850
3260,0.991607
3261,0.984017


In [ ]:
df['target'] = test_tar.apply(lambda row: 1 if row[0] > 0.5 else 0, axis=1)
df

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
df.to_csv('test_data.csv', index=False)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, Trainer, TrainingArguments

# Load and preprocess your dataset
# Make sure your dataset has two columns: 'text' and 'label'

# Load the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Replace '2' with the number of classes in your dataset

# Encode the dataset
def encode_examples(examples):
    inputs = tokenizer(examples['text'], padding='max_length', truncation=True, return_tensors='tf')
    return inputs, examples['label']

train_dataset = tf.data.Dataset.from_tensor_slices((dict(encode_examples(train_data)), train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((dict(encode_examples(val_data)), val_data))

# Define training arguments
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    save_steps=500,
    num_train_epochs=3,
    logging_dir="./logs",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=None,  # Use default data collator
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./saved_model")
